In [1]:
! pip install datasets ipywidgets transformers accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
 

## Imports

In [24]:
from typing import Dict, List, Optional, Union, Any, Tuple
from datasets import load_dataset, concatenate_datasets, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments, AutoModelForCausalLM, AutoModelForSequenceClassification
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
from scipy.stats import spearmanr
import numpy as np
from sklearn.model_selection import train_test_split
import gzip
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [2]:
torch.cuda.is_available()

True

## Tokenizer

The Tokenizer method, for now the maximum length is fixed to 512...

In [3]:
class CustomDataTokenizer:
    def __init__(self, tokenizer, is_classification=True, max_length = 512):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_classification = is_classification

    def __call__(self, data: Dict) -> Dict:
        # For Classification Task: Only Text1 is required...
        text_columns = ['text1']
        if not self.is_classification:
            text_columns.append('text2')

        tokens_list = []
        for text_column in text_columns:
            # Tokenization happens here to get in the form which is accepted in the Objective Function...
            tokens_list.append(self.tokenizer(data[text_column], max_length=self.max_length, truncation=True))

        token = {}
        seperate_ids = []
        for i, t in enumerate(tokens_list):
            for key, val in t.items():
                if i == 0:
                    token[key] = val
                else:
                    token[key] += val
                if key == 'input_ids':
                    seperate_ids += [i] * len(val)

        token['labels'] = [int(data['label']) if 'label' in data else -1]
        token['seperate_ids'] = seperate_ids

        return token

## Losses

The loss functions

In [4]:
def categorical_crossentropy(y_true: torch.Tensor, y_pred: torch.Tensor) -> torch.Tensor:
    return -(F.log_softmax(y_pred, dim=1) * y_true).sum(dim=1)

def cosine_loss(y_true: torch.Tensor, y_pred: torch.Tensor, tau: float = 20.0) -> torch.Tensor:
    y_true = y_true[::2, 0]
    y_true = (y_true[:, None] < y_true[None, :]).float()
    y_pred = F.normalize(y_pred, p=2, dim=1)
    y_pred = torch.sum(y_pred[::2] * y_pred[1::2], dim=1) * tau
    y_pred = y_pred[:, None] - y_pred[None, :]
    y_pred = (y_pred - (1 - y_true) * 1e12).view(-1)
    zero = torch.Tensor([0]).to(y_pred.device)
    y_pred = torch.concat((zero, y_pred), dim=0)
    return torch.logsumexp(y_pred, dim=0)

def angle_loss(y_true: torch.Tensor, y_pred: torch.Tensor, tau: float = 1.0):
    y_true = y_true[::2, 0]
    y_true = (y_true[:, None] < y_true[None, :]).float()

    y_pred_re, y_pred_im = torch.chunk(y_pred, 2, dim=1)
    a = y_pred_re[::2]
    b = y_pred_im[::2]
    c = y_pred_re[1::2]
    d = y_pred_im[1::2]

    z = torch.sum(c**2 + d**2, dim=1, keepdim=True)
    re = (a * c + b * d) / z
    im = (b * c - a * d) / z

    dz = torch.sum(a**2 + b**2, dim=1, keepdim=True)**0.5
    dw = torch.sum(c**2 + d**2, dim=1, keepdim=True)**0.5
    re /= (dz / dw)
    im /= (dz / dw)

    y_pred = torch.concat((re, im), dim=1)
    y_pred = torch.abs(torch.sum(y_pred, dim=1)) * tau
    y_pred = y_pred[:, None] - y_pred[None, :]
    y_pred = (y_pred - (1 - y_true) * 1e12).view(-1)
    zero = torch.Tensor([0]).to(y_pred.device)
    y_pred = torch.concat((zero, y_pred), dim=0)
    return torch.logsumexp(y_pred, dim=0)

def in_batch_negative_loss(y_true: torch.Tensor,
                           y_pred: torch.Tensor,
                           tau: float = 20.0,
                           negative_weights: float = 0.0) -> torch.Tensor:
    device = y_true.device

    def make_target_matrix(y_true: torch.Tensor):
        idxs = torch.arange(0, y_pred.shape[0]).int().to(device)
        y_true = y_true.int()
        idxs_1 = idxs[None, :]
        idxs_2 = (idxs + 1 - idxs % 2 * 2)[:, None]

        idxs_1 *= y_true.T
        idxs_1 += (y_true.T == 0).int() * -2

        idxs_2 *= y_true
        idxs_2 += (y_true == 0).int() * -1

        y_true = (idxs_1 == idxs_2).float()
        return y_true

    neg_mask = make_target_matrix(y_true == 0)

    y_true = make_target_matrix(y_true)

    y_pred = F.normalize(y_pred, dim=1, p=2)
    similarities = y_pred @ y_pred.T
    similarities = similarities - torch.eye(y_pred.shape[0]).to(device) * 1e12
    similarities = similarities * tau

    if negative_weights > 0:
        similarities += neg_mask * negative_weights

    return categorical_crossentropy(y_true, similarities).mean()

Combining the loss functions with weights

In [5]:
class TotalLoss:
    def __init__(self,
                w1: float = 1.0,
                w2: float = 1.0,
                w3: float = 1.0,
                cosine_tau: float = 20.0,
                ibn_tau: float = 20.0,
                angle_tau: float = 1.0):
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3
        self.cosine_tau = cosine_tau
        self.ibn_tau = ibn_tau
        self.angle_tau = angle_tau

    def __call__(self, labels: torch.Tensor, outputs: torch.Tensor) -> torch.Tensor:
        loss = 0.
        if self.w1 > 0:
            loss += self.w1 * cosine_loss(labels, outputs, self.cosine_tau)
        if self.w2 > 0:
            loss += self.w2 * in_batch_negative_loss(labels, outputs, self.ibn_tau)
        if self.w3 > 0:
            loss += self.w3 * angle_loss(labels, outputs, self.angle_tau)
        return loss

## Pooler

The different Pooling methods, using CLS for now, and Padding Strategy 'Left' for now

In [6]:
class Pooler:
    def __init__(self,
                model,
                # ['cls', 'cls_avg', 'last', 'avg', 'max', 'all', 'specific token index']
                pooling_strategy: Optional[Union[int, str]] = 'cls',
                padding_strategy: Optional[str] = 'left'):
        self.model = model
        self.pooling_strategy = pooling_strategy
        self.padding_strategy = padding_strategy

    def __call__(self, inputs) -> Any:
        if self.pooling_strategy == 'last':
            batch_size = inputs['input_ids'].shape[0]
            if self.padding_strategy == 'left':
                sequence_lengths = -1
            else:
                sequence_lengths = inputs["attention_mask"].sum(dim=1) - 1

        outputs = self.model(**inputs).last_hidden_state
        if self.pooling_strategy == 'cls':
            outputs = outputs[:, 0]
        elif self.pooling_strategy == 'cls_avg':
            outputs = (outputs[:, 0] + torch.mean(outputs, dim=1)) / 2.0
        elif self.pooling_strategy == 'last':
            outputs = outputs[torch.arange(batch_size, device=outputs.device), sequence_lengths]
        elif self.pooling_strategy == 'avg':
            outputs = torch.sum(
                outputs * inputs["attention_mask"][:, :, None], dim=1) / torch.sum(inputs["attention_mask"])
        elif self.pooling_strategy == 'max':
            outputs, _ = torch.max(outputs * inputs["attention_mask"][:, :, None], dim=1)
        elif self.pooling_strategy == 'all':
            return outputs
        elif isinstance(self.pooling_strategy, int) or self.pooling_strategy.isnumeric():
            return outputs[:, int(self.pooling_strategy)]
        return outputs

## Trainer

The custom trainer method which extends the Trainer method of Transformers

In [7]:
class CustomTrainer(Trainer):
    def __init__(self, pooler: Pooler, loss_kwargs: Optional[Dict] = None, **kwargs):
        super().__init__(**kwargs)
        self.pooler = pooler
        if loss_kwargs is None:
            loss_kwargs = {}
        self.loss_fct = TotalLoss(**loss_kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels", None)
        outputs = self.pooler(inputs)
        loss = self.loss_fct(labels, outputs)
        return (loss, outputs) if return_outputs else loss

## Data Collator

The custom data collator which works with the trainer

In [8]:
class CustomDataCollator:
    tokenizer = None
    padding = 'longest'
    max_length: Optional[int] = 512
    return_tensors: str = "pt"

    def __init__(self, tokenizer_base):
        self.tokenizer = tokenizer_base

    def __call__(self, features: List[Dict], return_tensors: str = "pt") -> Dict[str, torch.Tensor]:
        if return_tensors is None:
            return_tensors = self.return_tensors
        has_token_type_ids = "token_type_ids" in features[0]

        new_features = []
        for feature in features:
            seperate_ids = feature['seperate_ids']
            input_ids = feature['input_ids']
            attention_mask = feature['attention_mask']
            assert len(seperate_ids) == len(input_ids) == len(attention_mask)

            has_token_type_ids = False
            if "token_type_ids" in feature:
                has_token_type_ids = True
                token_type_ids = feature['token_type_ids']
                assert len(token_type_ids) == len(input_ids)

            max_seperate_id = max(seperate_ids)
            prev_start_idx = 0
            for seperate_id in range(1, max_seperate_id + 1):
                start_idx = seperate_ids.index(seperate_id)

                new_feature = {}
                new_feature['input_ids'] = input_ids[prev_start_idx:start_idx]
                new_feature['attention_mask'] = attention_mask[prev_start_idx:start_idx]
                if has_token_type_ids:
                    new_feature['token_type_ids'] = token_type_ids[prev_start_idx:start_idx]
                new_feature['labels'] = feature['labels']
                new_features.append(new_feature)
                prev_start_idx = start_idx

            new_feature = {}
            new_feature['input_ids'] = input_ids[prev_start_idx:]
            new_feature['attention_mask'] = attention_mask[prev_start_idx:]
            if has_token_type_ids:
                new_feature['token_type_ids'] = token_type_ids[prev_start_idx:]
            new_feature['labels'] = feature['labels']
            new_features.append(new_feature)

        del features
        features = self.tokenizer.pad(
            {'input_ids': [feature['input_ids'] for feature in new_features]},
            padding=self.padding,
            max_length=self.max_length,
            return_tensors=return_tensors,
        )
        features['attention_mask'] = self.tokenizer.pad(
            {'input_ids': [feature['attention_mask'] for feature in new_features]},
            padding=self.padding,
            max_length=self.max_length,
            return_tensors=return_tensors,
        )['input_ids']
        if has_token_type_ids:
            features['token_type_ids'] = self.tokenizer.pad(
                {'input_ids': [feature['token_type_ids'] for feature in new_features]},
                padding=self.padding,
                max_length=self.max_length,
                return_tensors=return_tensors,
            )['input_ids']
        features['labels'] = torch.Tensor([feature['labels'] for feature in new_features])

        return features

## Fit

The fit method which starts the training process, for now a lot of arguments have provided with default value...

In [9]:
def fit(train_ds,
        model_base,
        tokenizer_base,
        batch_size: int = 32,
        output_dir: Optional[str] = 'chk/new_c',
        epochs: int = 3,
        learning_rate: float = 1e-5,
        warmup_steps: int = 1000,
        logging_steps: int = 10,
        eval_steps: Optional[int] = None,
        save_steps: int = 100,
        save_strategy: str = 'steps',
        save_total_limit: int = 10,
        gradient_accumulation_steps: int = 1,
        fp16: Optional[bool] = None,
        argument_kwargs: Optional[Dict] = None,
        trainer_kwargs: Optional[Dict] = None,
        loss_kwargs: Optional[Dict] = None):

    if argument_kwargs is None:
        argument_kwargs = {}
    if trainer_kwargs is None:
        trainer_kwargs = {}
    callbacks = None

    pooler = Pooler(model_base)

    trainer = CustomTrainer(
        pooler=pooler,
        model=model_base,
        train_dataset=train_ds,
        loss_kwargs=loss_kwargs,
        tokenizer=tokenizer_base,
        args=TrainingArguments(
            per_device_train_batch_size=batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            warmup_steps=warmup_steps,
            num_train_epochs=epochs,
            learning_rate=learning_rate,
            fp16=fp16,
            logging_steps=logging_steps,
            save_strategy=save_strategy,
            eval_steps=eval_steps,
            save_steps=save_steps,
            output_dir=output_dir,
            save_total_limit=save_total_limit,
            load_best_model_at_end=False,
            ddp_find_unused_parameters=None,
            label_names=['labels', 'seperate_ids', 'extra'],
            **argument_kwargs,
        ),
        callbacks=callbacks,
        data_collator=CustomDataCollator(
            tokenizer_base
        ),
        **trainer_kwargs
    )

    trainer.train()
    return model_base, tokenizer_base, pooler

## Embeddings

The encode method to generate embeddings

In [10]:
def encode(inputs: Union[List[str], Tuple[str], List[Dict], str],
            model,
            pooler,
            tokenizer,
            max_length: Optional[int] = 512,
            to_numpy: bool = True,
            device: Optional[Any] = 'cuda:0'):
        if device is None:
            device = 'cpu'
        model.to(device)
        model.eval()

        tokens = tokenizer(
            inputs,
            padding='longest',
            max_length=max_length,
            truncation=True,
            return_tensors='pt')
        tokens.to(device)
        with torch.no_grad():
            output = pooler(tokens)
        if to_numpy:
            return output.float().detach().cpu().numpy()
        return output

# Execution

## Data Import

SentEval Datasets Import

In [ ]:
def get_senteval_binary_data(dataset):
    match dataset:
        case 'CR':
            pos_file = open("Data/custrev.pos", "r")
            neg_file = open("Data/custrev.neg", "r")
        case 'MPQA':
            pos_file = open("Data/mpqa.pos", "r")
            neg_file = open("Data/mpqa.neg", "r")
        case 'MR':
            pos_file = open("Data/rt-polarity.pos", "r")
            neg_file = open("Data/rt-polarity.neg", "r")
        case 'SUBJ':
            pos_file = open("Data/subj.objective", "r")
            neg_file = open("Data/subj.subjective", "r")

    df_pos = pd.DataFrame()
    pos_content = pos_file.readlines()
    pos_file.close()
    df_pos['sentence'] = pos_content
    labels = np.ones(len(pos_content))
    df_pos['label'] = labels.astype('int')

    df_neg = pd.DataFrame()
    neg_content = neg_file.readlines()
    neg_file.close()
    df_neg['sentence'] = neg_content
    labels = np.zeros(len(neg_content))
    df_neg['label'] = labels.astype('int')

    df = pd.concat([df_pos, df_neg], axis=0, ignore_index=True)
    df['sentence'] = df['sentence'].str.replace('\n', '')
    return df

In [ ]:
# SST -> We use the Binary classification...
def get_sst_data():
    train_file = open("Data/sentiment-train", "r")
    train = train_file.readlines()
    train_file.close()

    sentence_train = []
    label_train = []
    for sentence in train:
        sentence = sentence.strip()
        label = int(sentence[len(sentence) - 1])
        sentence = sentence[:-1].strip()
        sentence_train.append(sentence)
        label_train.append(label)

    df_train = pd.DataFrame({'sentence': sentence_train, 'label': label_train})

    test_file = open("Data/sentiment-test", "r")
    test = test_file.readlines()
    test_file.close()

    sentence_test = []
    label_test = []
    for sentence in test:
        sentence = sentence.strip()
        label = int(sentence[len(sentence) - 1])
        sentence = sentence[:-1].strip()
        sentence_test.append(sentence)
        label_test.append(label)

    df_test = pd.DataFrame({'sentence': sentence_test, 'label': label_test})
    df = pd.concat([df_train, df_test], axis=0, ignore_index=True)
    return df

In [ ]:
def get_senteval_dataset(name):
    match name:
        case 'CR':
            return get_senteval_binary_data('CR')
        case 'MPQA':
            return get_senteval_binary_data('MPQA')
        case 'MR':
            return get_senteval_binary_data('MR')
        case 'SST':
            return get_sst_data()
        case 'SUBJ':
            return get_senteval_binary_data('SUBJ')

In [11]:
# senteval_datasets = ['CR', 'MPQA', 'MR', 'SUBJ']#, 'SST']
senteval_datasets = ['DATA']

In [12]:
def get_custom_data():
  benign_data = pd.read_csv("/content/drive/MyDrive/Thesis_Projects/URL_detection/Benign.csv")

  malicious_data = pd.read_csv("/content/drive/MyDrive/Thesis_Projects/URL_detection/Malicious.csv")

  df1 = pd.DataFrame(benign_data)
  df2 = pd.DataFrame(malicious_data)

  x = df1.sample(25000)
  y = df2.sample(25000)

  data = pd.concat([x,y], axis=0)
  data = shuffle(data)
  data.rename(columns={'url': 'sentence'}, inplace=True)
  return data

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


STS Datasets Import

In [ ]:
def get_sts_dataset(dataset_name):
    match dataset_name:
        case 'STS-B':
            dataset = load_dataset('mteb/stsbenchmark-sts', split='test')
        case 'STS12':
            dataset = load_dataset('mteb/sts12-sts', split='test')
        case 'STS13':
            dataset = load_dataset('mteb/sts13-sts', split='test')
        case 'STS14':
            dataset = load_dataset('mteb/sts14-sts', split='test')
        case 'STS15':
            dataset = load_dataset('mteb/sts15-sts', split='test')
        case 'STS16':
            dataset = load_dataset('mteb/sts16-sts', split='test')
        case 'SICK-R':
            dataset = load_dataset('mteb/sickr-sts', split='test')
    return dataset

In [ ]:
sts_datasets = ['STS-B', 'STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'SICK-R']

## Objective Function Combinations

Using the 3 provided objective functions and their 7 possible combinations for now

In [14]:
def get_objective_function_weights(combi):
    match combi:
        case 'Cosine': return (1, 0, 0)
        case 'In-Batch Negatives': return (0, 1, 0)
        case 'Angle': return (0, 0, 1)
        case 'Cosine + In-Batch Negatives': return (1, 1, 0)
        case 'Cosine + Angle': return (1, 0, 1)
        case 'In-Batch Negatives + Angle': return (0, 1, 1)
        case 'Cosine + In-Batch Negatives + Angle': return (1, 1, 1)

In [22]:
objective_functions = [
    # 'Cosine',
    # 'In-Batch Negatives',
    'Angle',
    # 'Cosine + In-Batch Negatives',
    # 'Cosine + Angle',
    # 'In-Batch Negatives + Angle',
    # 'Cosine + In-Batch Negatives + Angle'
]

## Language Models

Base Model Selection

In [16]:
models = [
    # 'bert-base-uncased',
    # 'bert-base-cased',
    # 'bert-large-uncased',
    # 'bert-large-cased'
    # 'FacebookAI/roberta-base',
    # 'sentence-transformers/all-mpnet-base-v2',
    'princeton-nlp/sup-simcse-roberta-large'
]

## Driver Functions

### SentEval

In [25]:
def driver_senteval():
    results_matrix = []

    for model in models:
        results_obj_matrix = []
        for objective in objective_functions:
            # Objective Functions Preparation...
            w1_combi, w2_combi, w3_combi = get_objective_function_weights(objective)
            results_obj_ds_matrix = []

            for dataset in senteval_datasets:
                # Model Preparation...
                tokenizer_base = AutoTokenizer.from_pretrained(model)
                model_base = AutoModelForSequenceClassification.from_pretrained(model)

                # Dataset Preparation...
                # df = get_senteval_dataset(dataset)
                df = get_custom_data()
                ds = Dataset.from_pandas(df)
                ds = ds.rename_column('sentence', 'text1')
                if dataset == 'MR':
                    ds = concatenate_datasets([ds.select(range(0, 231)), ds.select(range(233, 7463))])

                split_ds = ds.train_test_split(test_size=0.3, seed=42)
                ds_train = split_ds['train']
                ds_test = split_ds['test']

                # Tokenization...
                train_ds = ds_train.shuffle().map(CustomDataTokenizer(tokenizer_base), num_proc=8)

                # Model Training...
                model_new, tokenizer_new, pooler_new = fit(
                    train_ds=train_ds,
                    model_base=model_base,
                    tokenizer_base=tokenizer_base,
                    output_dir='chk/c',
                    batch_size=20,
                    epochs=5,
                    learning_rate=2e-5,
                    save_steps=0,
                    eval_steps=100,
                    warmup_steps=0,
                    gradient_accumulation_steps=1,
                    loss_kwargs={
                        'w1': w1_combi,
                        'w2': w2_combi,
                        'w3': w3_combi,
                        'cosine_tau': 20,
                        'ibn_tau': 20,
                        'angle_tau': 1.0
                    },
                    fp16=True,
                    logging_steps=1000
                )

                # Embedding Generation for Train and Test sets... Doing line-by-line embeddings for now...
                emb_train = []
                for sentence in ds_train['text1']:
                    emb_train.append(encode(sentence, model_new, pooler_new, tokenizer_new)[0])

                emb_test = []
                for sentence in ds_test['text1']:
                    emb_test.append(encode(sentence, model_new, pooler_new, tokenizer_new)[0])

                # Conversion into Numpy Array...
                emb_train = np.array(emb_train)
                emb_test = np.array(emb_test)

                # Classification...
                # lr = LogisticRegression(max_iter=10000)
                lr = SVC()
                lr.fit(emb_train, ds_train['label'])
                results_obj_ds_matrix.append(lr.score(emb_test, ds_test['label']))
            results_obj_matrix.append(results_obj_ds_matrix)
        results_matrix.append(results_obj_matrix)
    return results_matrix

### STS

In [ ]:
def calculate_cosine_similarity(sentence1_vec, sentence2_vec):
    cosine_similarity = np.dot(sentence1_vec, sentence2_vec) / (np.linalg.norm(sentence1_vec) * np.linalg.norm(sentence2_vec))
    return cosine_similarity

In [ ]:
def calculate_Spearman_rank_correlation_coefficient(scores, scores_actual):
    sc, _ = spearmanr(scores, scores_actual)
    return sc

In [ ]:
def driver_sts():
    results_matrix = []

    for model in models:
        results_obj_matrix = []
        for objective in objective_functions:
            # Objective Functions Preparation...
            w1_combi, w2_combi, w3_combi = get_objective_function_weights(objective)
            results_obj_ds_matrix = []

            for dataset in sts_datasets:
                # Model Preparation...
                tokenizer_base = AutoTokenizer.from_pretrained(model)
                model_base = AutoModel.from_pretrained(model)

                # Dataset Preparation...
                ds = get_sts_dataset(dataset)
                ds = ds.rename_column('sentence1', 'text1')
                ds = ds.rename_column('sentence2', 'text2')
                ds = ds.rename_column('score', 'label')

                split_ds = ds.train_test_split(test_size=0.3, seed=42)
                ds_train = split_ds['train']
                ds_test = split_ds['test']

                # Tokenization of train dataset...
                train_ds = ds_train.shuffle().map(CustomDataTokenizer(tokenizer_base, is_classification=False), num_proc=8)

                # Model Training...
                model_new, tokenizer_new, pooler_new = fit(
                    train_ds=train_ds,
                    model_base=model_base,
                    tokenizer_base=tokenizer_base,
                    output_dir='chk/c',
                    batch_size=32,
                    epochs=5,
                    learning_rate=2e-5,
                    save_steps=0,
                    eval_steps=100,
                    warmup_steps=0,
                    gradient_accumulation_steps=1,
                    loss_kwargs={
                        'w1': w1_combi,
                        'w2': w2_combi,
                        'w3': w3_combi,
                        'cosine_tau': 20,
                        'ibn_tau': 20,
                        'angle_tau': 1.0
                    },
                    fp16=True,
                    logging_steps=1000
                )

                # Generating embeddings of STS dataset using the newly trained model...
                emb_sentence_1 = encode(ds_test['text1'], model_new, pooler_new, tokenizer_new) # generating embeddings for test set sentence 1
                emb_sentence_2 = encode(ds_test['text2'], model_new, pooler_new, tokenizer_new) # generating embeddings for test set sentence 2

                # Calculating Spearman for AnglE...
                cos_score = []
                for i in range(emb_sentence_1.shape[0]):
                    cos_score.append(calculate_cosine_similarity(emb_sentence_1[i], emb_sentence_2[i]))

                spearman = calculate_Spearman_rank_correlation_coefficient(cos_score, ds_test['label'])
                results_obj_ds_matrix.append(spearman)
            results_obj_matrix.append(results_obj_ds_matrix)
        results_matrix.append(results_obj_matrix)
    return results_matrix

### Running

In [ ]:
# results_matrix_sts = driver_sts()

In [26]:
results_matrix_senteval = driver_senteval()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=8):   0%|          | 0/35000 [00:00<?, ? examples/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


AttributeError: 'SequenceClassifierOutput' object has no attribute 'last_hidden_state'

### Saving the Results

In [20]:
results_matrix_senteval

[[[0.49566666666666664]]]

In [ ]:
with open('BERT_SentEval_Results.npy', 'wb') as f:
    np.save(f, results_matrix_senteval)

In [ ]:
with open('BERT_STS_Results.npy', 'wb') as f:
    np.save(f, results_matrix_sts)

# NLI

In [ ]:
def load_all_nli(exclude_neutral=True):
    label_mapping = {
        'entailment': 1,  # '0' (entailment)
        'neutral': 1,
        'contradiction': 0   # '2' (contradiction)
    }
    data = []
    with gzip.open('AllNLI.tsv.gz', 'rt', encoding='utf8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            if row['split'] == 'train' and row['label'] != 'neutral':
                if exclude_neutral and row['label'] == 'neutral':
                    continue
                sent1 = row['sentence1'].strip()
                sent2 = row['sentence2'].strip()
                data.append({'text1': sent1, 'text2': sent2, 'label': label_mapping[row['label']]})
    return data

In [ ]:
def preprocess_nli():
    train_data = load_all_nli()
    nli_dataset = {}
    train_ds = Dataset.from_list(train_data)
    nli_dataset['train'] = train_ds
    nli_dataset = DatasetDict(nli_dataset)
    ds_train = nli_dataset['train']
    return ds_train

Training with AnglE losses...

In [ ]:
def train_nli(ds_train, model_base, tokenizer_base):
    train_ds = ds_train.shuffle().map(CustomDataTokenizer(tokenizer_base), num_proc=8)
    model_new, tokenizer_new, pooler_new = fit(
        train_ds=train_ds,
        model_base=model_base,
        tokenizer_base=tokenizer_base,
        output_dir='chk/c',
        batch_size=32,
        epochs=5,
        learning_rate=2e-5,
        save_steps=0,
        eval_steps=100,
        warmup_steps=0,
        gradient_accumulation_steps=1,
        loss_kwargs={
            'w1': 1.0,
            'w2': 1.0,
            'w3': 1.0
            'cosine_tau': 20,
            'ibn_tau': 20,
            'angle_tau': 1.0
        },
        fp16=True,
        logging_steps=1000
    )
    return model_new, tokenizer_new, pooler_new